In [42]:
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Set up display
width, height = 840, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("El Examen")

# Colors
white = (255, 255, 255)
black = (0, 0, 0)

#Font
font = pygame.font.Font(None,32)
font_2 = pygame.font.Font(None,25)

#Buttons
button_width, button_height = 120, 50

button_color = black
button_math = pygame.Rect(120, 50, button_width, button_height)
button_chem = pygame.Rect(360, 50, button_width, button_height)
button_eng = pygame.Rect(600, 50, button_width, button_height)

# Create a clock object to control the frame rate
clock = pygame.time.Clock()
fps = 60  # Set the desired frames per second

running = True

def button_dynamics(button, name, subname_1, subname_2, subname_3):
        hover = button.collidepoint(mouse_x, mouse_y)

        if hover and button.height < 345:
            button.height += 5
        elif not hover and button.height > 50:
            button.height -= 5
        
        button = pygame.Rect(button.x, button.y, button.width, button.height)
        pygame.draw.rect(screen, black, button,0, 30)

        text = font.render(name, True, white)
        text_button = text.get_rect(center=(button.centerx, button.y+25))
        screen.blit(text, text_button)

        if button.height > 110:
            subbutton_1 = pygame.Rect(button.x+10,button.y+60,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_1,0, 30)
            text_1 = font_2.render(subname_1, True, black)
            text_subbutton_1 = text_1.get_rect(center=subbutton_1.center)
            screen.blit(text_1, text_subbutton_1)

        if button.height > 210:
            subbutton_2 = pygame.Rect(button.x+10,button.y+160,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_2,0, 30)
            text_2 = font_2.render(subname_2, True, black)
            text_subbutton_2 = text_2.get_rect(center=subbutton_2.center)
            screen.blit(text_2, text_subbutton_2)

        if button.height > 310:
            subbutton_3 = pygame.Rect(button.x+10,button.y+260,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_3,0, 30)
            text_3 = font_2.render(subname_3, True, black)
            text_subbutton_3 = text_3.get_rect(center=subbutton_3.center)
            screen.blit(text_3, text_subbutton_3)

while running:
    screen.fill(white)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    mouse_x, mouse_y = pygame.mouse.get_pos()

    button_dynamics(button_math, 'Math','Game 1','Game 2','Game 3')
    button_dynamics(button_chem, 'Chemistry','Game 1','Game 2','Game 3')
    button_dynamics(button_eng,'English','Game 1','Game 2','Game 3')

# Update the display
    pygame.display.flip()

    # Cap the frame rate to 60 fps
    clock.tick(fps)

# Quit Pygame
pygame.quit()
sys.exit()

SystemExit: 